# Imports

In [48]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm

from sympy import Symbol, solve # 방정식 해 구하기

# Data settings

## Load dataset
* 데이터셋 불러오기
* `net` : 네트워크 파일
* `OD` : Origin-Destination 수요

In [49]:
data_dir = 'dataset'

In [50]:
network_file = 'Network.csv'
network_path = os.path.join(data_dir, network_file)

In [51]:
OD_file = 'OD.csv'
OD_path = os.path.join(data_dir, OD_file)

In [52]:
network = pd.read_csv(network_path, encoding = 'UTF-8')

In [53]:
OD = pd.read_csv(OD_path, encoding = 'UTF-8')

In [54]:
network.head(3)
#> Init node

,Init node,Term node,Capacity,Length,Free Flow Time,B,Power,Speed limit,Toll,Type
0,1,2,25900.20064,6,6,0.15,4,0,0,1
1,1,3,23403.47319,4,4,0.15,4,0,0,1
2,2,1,25900.20064,6,6,0.15,4,0,0,1


In [55]:
OD.tail(20).T

,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575
O,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24
D,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
T,0,100,100,200,200,800,600,500,700,400,400,300,300,0,100,400,500,1100,700,0


## Preprocessing
* 필요 변수
    * `O` : 기점
    * `D` : 종점
    * `Link_num` : 링크번호
    * `FFT` : Free Flow Time
    * `Alpha`, `Beta` : BPR 함수에 필요한 상수들
    * `Length` : 네트워크 길이
    * `Toll` : 네트워크 요금
    * `Volume` : 링크 통행량
* `BPR Function` : `Cost = Free Flow Time * (1 + (Alpha * (Xn / Capacity))**Beta) + Length * Toll`

In [56]:
def preprocessing(network):
    """네트워크 객체의 컬럼 이름 및 순서를 정리해줌
    - O, D, FFT, alpha, Capacity, beta, Length, Toll 만 남기고 나머지는 지운다.
    """
    # cost를 구한다.
    net = pd.DataFrame({
        'O' : network['Init node '],
        'D' : network['Term node '],
        'FFT' : network['Free Flow Time '],
        'Power' : network['Power'],
        'Capacity' : network['Capacity '],
        'B' : network['B'],
        'Length' : network['Length '],
        #'Toll' : network['Toll '],
        'Link_num' : [i for i in range(1, len(network)+1)],
        'x_n' : 0,
        'y_n' : 0,
        'Direction' : 0,
        'x_n1' : 0,
        'Cost' : 0
    })
   
    net = net.sort_values(by = ['O', 'D']) # net1 데이터프레임을 O, D에 대하여 오름차순 정렬
    net.set_index('Link_num', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
    
    return net

# Shortest Path Definition
* `Dijkstra` 방식을 사용하여 각 O-D별 최단경로를 추출하는 함수 생성

## Node Range

In [57]:
def node_range(network):
    """Sioux-Falls Network의 각 기점(Origin)에서 출발하는 링크 번호의 범위를 데이터프레임에 저장하는 함수"""
    origin_nodes = network['O'].unique()
    start_list = []
    end_list = []
    
    for origin in origin_nodes:
        link_list = network[network['O'] == origin]['Link_num'] # 기점이 origin인 모든 링크 번호 리스트 추출
        link_list.sort_values() # 기점 origin에 연결된 링크 번호들을 오름차순으로 정렬
        
        start = link_list.iloc[0] # 링크 리스트의 첫번째 링크(가장 먼저 번호 링크)
        end = link_list.iloc[-1] # 링크 리스트의 마지막 링크(가장 마지막 번호 링크)
        
        start_list.append(start)
        end_list.append(end)
        
    node_link_df = pd.DataFrame({
        'origin' : origin_nodes,
        'start_link' : start_list,
        'end_link' : end_list
    })
    
    node_link_df.set_index('origin', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
    
    return node_link_df

## Calculate Cost : $t_a(x_a)$
* 매 iteration마다 갱신되는 $\alpha$, $x_{n}$ 값을 이용해서 Cost를 계산해 주는 함수
* 따라서 iteration을 돌 때마다 다시 계산해주어야 한다.

In [58]:
def calculate_cost(network):
    """
    BPR 함수를 사용하여 링크 통행비용을 계산하는 함수
    :: 필요 변수 :: FFT, Power, B, X0, Capacity, Length, Toll
    :: BPR 함수 :: Cost = Free Flow Time * (1 + (B * (X_n / Capacity))**Power) + length * toll
    """
    network['Cost'] = network['FFT'] * (1 + network['B'] * ((network['x_n'] / network['Capacity'])**network['Power'])) #+ (net1['Length'] * net1['Toll'])
    
    return network

## Shortest Path Algorithm : Dijkstra

In [59]:
def tree_Dijkstra(origin, network):
    """Dijkstra 방법을 이용해 Shortest Path를 구하기 위한 경로 트리를 구해주는 과정.
    이때 network는 반드시 cost 컬럼을 포함하고 있어야 합니다."""
    
    SP_df = pd.DataFrame({
        'To' : network['O'].unique(),
        'From' : 0,
        'C1' : 999,
        'Fin' : 0
    })
    
    SP_df.loc[SP_df['To'] == origin, 'C1'] = 0 # 'To' 열 값이 origin과 같은 행의 C1 열 값을 0으로 치환하기
    SP_df = SP_df.sort_values(by = ['C1', 'To']).reset_index(drop = True) # cost_df 를 C1과 To값에 따라서 정렬하기
    
    node_link = node_range(network) # 각 origin을 기점으로 하는 링크번호 범위 데이터프레임
    
    while True:
        if sum(SP_df['Fin']) == len(SP_df):
            break

        a = SP_df.loc[0, 'To'] # cost_df의 1행 1열 데이터
        ac = SP_df.loc[0, 'C1'] # cost_df의 1행 3열 데이터
        
        SP_df.loc[0, 'Fin'] = 1
        SP_df = SP_df.sort_values(by = ['To']).reset_index(drop = True)
            
        # DN : Dijkstra next updating node, D : Dijkstra iteration term
        node_a = node_link[node_link['origin'] == a] # node_range의 링크 중, origin이 a인 행만 추출
        start_link_a = node_a['start_link'].iloc[0] # origin a에 연결된 link들 중 첫번째 번호
        end_link_a = node_a['end_link'].iloc[0] # origin a에 연결된 link들 중 마지막 번호
            
        DN = network[(network['Link_num'] >= start_link_a) & (network['Link_num'] <= end_link_a)]
        
        DN = DN[['D', 'Cost']]
        DN.rename(columns = {'D' : 'To', 'Cost' : 'C'}, inplace = True)
            
        SP_df = pd.merge(SP_df, DN, on = ['To'], how = 'left') 
            
        SP_df = SP_df.fillna(999) # SP_df의 C 값 중 값이 없어 NaN인 값들을 999로 채우기
        #SP_df = SP_df.sort_values(by = 'To').reset_index(drop = True)
            
        upset = list(SP_df[SP_df['C1'] > (SP_df['C'] + ac)]['To']) ######### 진짜 여기서 죽는줄알았음 헛갈림 주의
        
        #print(upset)
        #print(Pk)
            
        if len(upset) > 0:
            SP_df.loc[SP_df['To'].isin(upset), 'Fin'] = 0
            SP_df.loc[SP_df['To'].isin(upset), 'C1'] = SP_df.loc[SP_df['To'].isin(upset), 'C'] + ac
            SP_df.loc[SP_df['To'].isin(upset), 'From'] = a
            
        SP_df = SP_df.iloc[:, 0:-1]
        SP_df = SP_df.sort_values(by = ['Fin', 'C1', 'To']).reset_index(drop = True)
    
    SP_output = pd.DataFrame({
        'O' : SP_df['From'],
        'D' : SP_df['To'],
        'SP' : SP_df['C1']
    })
    
    SP_output = SP_output.sort_values(by = 'SP').reset_index(drop = True)
    
    return SP_output

In [60]:
def SPD_Dijkstra(origin, destination, network):
    """주어진 네트워크 및 비용(SP)을 바탕으로,
    origin >> destination으로 향하는 최단 경로를 구해주는 함수.
    network 데이터프레임은 반드시 cost 열을 포함하고 있어야 합니다."""
    
    mintree = tree_Dijkstra(origin, network)
    OD_SP = mintree[mintree['D'] == destination]
    
    while True:
        if OD_SP['O'].iloc[0] == origin:
            break
        
        added = mintree[mintree['D'] == OD_SP['O'].iloc[0]]
        OD_SP = pd.concat([added, OD_SP], ignore_index = True)
        
    # 각 O-D에 Node Link 붙이기 : network의 Link_num 이용
    OD_SP = pd.merge(OD_SP, network, on = ['O', 'D'], how = 'left')
    OD_SP = OD_SP[['O', 'D', 'SP', 'Link_num']]
    OD_SP.set_index('Link_num', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
        
    return OD_SP

# Initialization
* **`N = 0`**
* 초기 비용 계산 : $x_0$ = 0 (모든 링크의 통행량 = 0)인 상태에서 비용 계산
* 초기 배정 : All-or-Nothing 방식으로 통행 배정, $x_1$ 획득

## Data Ready

In [61]:
net1 = preprocessing(network)

In [62]:
net1.head(3)

,O,D,FFT,Power,Capacity,B,Length,Toll,Link_num,x_n,y_n
Link_num,,,,,,,,,,,
1,1,2,6,4,25900.20064,0.15,6,0,1,0,0
2,1,3,4,4,23403.47319,0.15,4,0,2,0,0
3,2,1,6,4,25900.20064,0.15,6,0,3,0,0


## 초기 비용 계산 : $t_a(x_a^0)$
* $x_0$ = 0 (전부 0)임을 계산하여 Cost 계산

In [63]:
net_cost = calculate_cost(net1) # x0 

In [64]:
net_cost.head()

,O,D,FFT,Power,Capacity,B,Length,Toll,Link_num,x_n,y_n,Cost
Link_num,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,0,1,0,0,6.0
2,1,3,4,4,23403.473190,0.15,4,0,2,0,0,4.0
3,2,1,6,4,25900.200640,0.15,6,0,3,0,0,6.0
4,2,6,5,4,4958.180928,0.15,5,0,4,0,0,5.0
5,3,1,4,4,23403.473190,0.15,4,0,5,0,0,4.0


## $x_1$ 획득 : AON Assignment

In [ ]:
# 각 OD별로 Dijkstra를 사용해 최단경로를 산출, 
# OD 테이블의 교통량(T)을 각 최단경로에 포함된 링크에 All-or-Nothing으로 배정하기

origin_list = OD['O'].unique()

for origin in tqdm(origin_list): # 각 Origin에 대하여
    
    #print(f'-------- origin = {origin} --------')
    
    each_origin_OD = OD[OD['O'] == origin]
    destination_list = each_origin_OD['D'].unique()
    
    for destination in destination_list: # 각 Origin의 Destin에 대하여
        # OD별 shortest path 산출, 링크 넘버 추출
        if origin != destination:
            SPD = SPD_Dijkstra(origin, destination, net_cost)
            link_list = list(SPD['Link_num'])
            #print(f'(O, D) = ({origin}, {destination}), Path link list = {link_list}')
            # 각 링크넘버에 OD 테이블의 교통량(T) 더해주기
            add_volume = OD[(OD['O'] == origin) & (OD['D'] == destination)].iloc[0].loc['T'] # 각 통과 링크에 더해줄 OD 교통량 산정 
            net_cost.loc[net_cost['Link_num'].isin(link_list), 'x_n'] += add_volume # All or Nothing 배정
        else:
            pass

In [66]:
net_cost

,O,D,FFT,Power,Capacity,B,Length,Toll,Link_num,x_n,y_n,Cost
Link_num,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,0,1,3800,0,6.0
2,1,3,4,4,23403.473190,0.15,4,0,2,6000,0,4.0
3,2,1,6,4,25900.200640,0.15,6,0,3,3800,0,6.0
4,2,6,5,4,4958.180928,0.15,5,0,4,6600,0,5.0
5,3,1,4,4,23403.473190,0.15,4,0,5,6000,0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
72,23,22,4,4,5000.000000,0.15,4,0,72,10800,0,4.0
73,23,24,2,4,5078.508436,0.15,2,0,73,5700,0,2.0
74,24,13,4,4,5091.256152,0.15,4,0,74,10800,0,4.0


# Iteration

## $t_1$ 계산 : 통행비용 갱신
* Update Link Travel Time
* 전 단계에서 계산한 통행량 $x_1$로 통행비용(Cost) 갱신

In [67]:
net_cost = calculate_cost(net_cost) # Cost 일부 갱신

In [68]:
net_cost.head()

,O,D,FFT,Power,Capacity,B,Length,Toll,Link_num,x_n,y_n,Cost
Link_num,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,0,1,3800,0,6.000417
2,1,3,4,4,23403.473190,0.15,4,0,2,6000,0,4.002592
3,2,1,6,4,25900.200640,0.15,6,0,3,3800,0,6.000417
4,2,6,5,4,4958.180928,0.15,5,0,4,6600,0,7.354765
5,3,1,4,4,23403.473190,0.15,4,0,5,6000,0,4.002592


## $y_1$ 획득 : AON Assignment
* 전 단계의 $t_1$을 이용, Dijkstra로 Shortest Path 계산
* All or Nothing Assignment 실행, y1 획득

In [69]:
for origin in tqdm(origin_list): # 각 Origin에 대하여
    
    #print(f'origin = {origin}')
    each_origin_OD = OD[OD['O'] == origin]
    destination_list = each_origin_OD['D'].unique()
    
    for destination in destination_list: # 각 Origin의 Destin에 대하여
        # OD별 shortest path 산출, 링크 넘버 추출
        if origin != destination:
            SPD = SPD_Dijkstra(origin, destination, net_cost)
            link_list = list(SPD['Link_num'])
            #print(f'destination = {destination}, link list = {link_list}')
            # 각 링크넘버에 OD 테이블의 교통량(T) 더해주기
            add_volume = OD[(OD['O'] == origin) & (OD['D'] == destination)].iloc[0].loc['T'] # 각 통과 링크에 더해줄 OD 교통량 산정 
            net_cost.loc[net_cost['Link_num'].isin(link_list), 'y_n'] += add_volume # All or Nothing 배정 :: y0 갱신
        else:
            pass

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [01:35<00:00,  3.98s/it]


In [70]:
net_cost

,O,D,FFT,Power,Capacity,B,Length,Toll,Link_num,x_n,y_n,Cost
Link_num,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,0,1,3800,4800,6.000417
2,1,3,4,4,23403.473190,0.15,4,0,2,6000,12900,4.002592
3,2,1,6,4,25900.200640,0.15,6,0,3,3800,4900,6.000417
4,2,6,5,4,4958.180928,0.15,5,0,4,6600,1600,7.354765
5,3,1,4,4,23403.473190,0.15,4,0,5,6000,12800,4.002592
...,...,...,...,...,...,...,...,...,...,...,...,...
72,23,22,4,4,5000.000000,0.15,4,0,72,10800,7300,17.060694
73,23,24,2,4,5078.508436,0.15,2,0,73,5700,15900,2.476076
74,24,13,4,4,5091.256152,0.15,4,0,74,10800,11900,16.149166


## Find Move Size $\alpha$
* Finding Direction : $d = y_1-x_1$ 
* 목적함수 $Z(x)$를 최소화하는 $\alpha$ 값 찾기
* 즉, $\sum(y_a^n-x_a^n)*t_a*(x_a^n+\alpha(y_a^n-x_a^n) = 0$이 되는 $\alpha$ 값 찾기

In [71]:
net_cost['Direction'] = net_cost['y_n'] - net_cost['x_n'] # direction :: d = yn - xn 임

In [72]:
net_cost['Direction'].unique()

array([  1000,   6900,   1100,  -5000,   6800,  22100,  15600,  20600,
        22500,   4400,  20300,  -4200,  35500,  -4900,  -5100, -16800,
        14100,   5900, -17600,  14700,  23000, -14700,  34200,  23600,
        18500,  17800, -10800,  42200, -28800,  23400,   5100,  -9200,
       -11300, -16900,  17000,   3600,   5000,   -300, -14600,  10000,
        -8400,  39200,  -4400,  14900,  -2600, -15500, -28100, -26700,
        11400, -21900,   5300,  11300,  12600,  20100,  27700,  11900,
        32900,   4800,  12300,   6700, -13000, -12800, -25200,  16500,
         8300,  -3500,  10200, -10900], dtype=int64)

In [73]:
## Sympy :: 방정식을 풀기 위한 패키지 이용. 방정식에 포함될 각 변수들을 선언하기

d = Symbol('d') # direction
B = Symbol('B') # B. 0.15 있는 그거
t = Symbol('t') # t. Cost
c = Symbol('c') # capacity
x = Symbol('x') # x0
alpha = Symbol('alpha') # alpha
P = Symbol('P') # Power. 4.

In [74]:
net_cost.head(3)

,O,D,FFT,Power,Capacity,B,Length,Toll,Link_num,x_n,y_n,Cost,Direction
Link_num,,,,,,,,,,,,,
1,1,2,6,4,25900.20064,0.15,6,0,1,3800,4800,6.000417,1000
2,1,3,4,4,23403.47319,0.15,4,0,2,6000,12900,4.002592,6900
3,2,1,6,4,25900.20064,0.15,6,0,3,3800,4900,6.000417,1100


In [75]:
total_equation = 0 # total Equation : Z(x)가 최소화되기 위한 조건식

for i in tqdm(range(1, len(net_cost)+1)): # 1부터 끝까지 Link_num == i에 대하여
    
    d = net_cost.loc[i, 'Direction'] # i링크의 direction
    t = net_cost.loc[i, 'Cost']
    B = net_cost.loc[i, 'B']
    x = net_cost.loc[i, 'x_n']
    P = net_cost.loc[i, 'Power']
    C = net_cost.loc[i, 'Capacity']
    
    equation = d * t * (1 + B * (((x + alpha * d)/C)**P))
    #print(equation)
    
    total_equation += equation

100%|█████████████████████████████████████████████████████████████████████████████████| 76/76 [00:00<00:00, 294.82it/s]


In [76]:
alpha_list = solve(total_equation) # 여기서 alpha는 0과 1 사이의 양수여야 하므로, 해당 부분만을 필터링해준다.
print(alpha_list)

[0.769414695789773, 1.59442500697876, 0.852668536610942 - 0.326989551972665*I, 0.852668536610942 + 0.326989551972665*I]


In [77]:
def alpha_range(x):
    """Alpha 값 중 0 이상 1 이하의 값만 반환하게끔 하는 함수"""
    decision = (x <= 1) and (x >= 0)
    return decision

In [78]:
def find_alpha(alpha_list):
    """Alpha 값들 중 0 이상 1 이하 실수값만을 반환"""
    for alpha in alpha_list:
        if alpha.is_real == True:
            if alpha <= 1 and alpha >= 0:
                #print(alpha)
                return alpha
            else:
                pass
        else:
            pass

In [79]:
ith_alpha = find_alpha(alpha_list)
print(ith_alpha) #### 금번 iteration의 alpha를 구했다.

0.769414695789773


## $x_2$ 계산
* $\alpha$ 값을 이용하여 $x_2$ 값을 계산
* $x_2 = x_1 + \alpha*(y_1 - x_1)$

In [80]:
net_cost['x_n1'] = net_cost['x_n'] + ith_alpha * (net_cost['y_n'] - net_cost['x_n'])

In [81]:
net_cost.head()

,O,D,FFT,Power,Capacity,B,Length,Toll,Link_num,x_n,y_n,Cost,Direction,x_n1
Link_num,,,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,0,1,3800,4800,6.000417,1000,4569.41469578977
2,1,3,4,4,23403.473190,0.15,4,0,2,6000,12900,4.002592,6900,11308.9614009494
3,2,1,6,4,25900.200640,0.15,6,0,3,3800,4900,6.000417,1100,4646.35616536875
4,2,6,5,4,4958.180928,0.15,5,0,4,6600,1600,7.354765,-5000,2752.92652105114
5,3,1,4,4,23403.473190,0.15,4,0,5,6000,12800,4.002592,6800,11232.0199313705


## Convergence Test
* 컨버전스 테스트 실행
* 만약 컨버전스 테스트를 통과하지 못할 시, 이 $x_2$를 이용해서 iteration을 반복하게 된다.

In [86]:
def convergence_test(network, k):
    """네트워크에 대한 convergence test를 실행하는 함수.
    :: network :: 네트워크 데이터프레임. x_n, x_n1, Cost 열을 포함하고 있어야 함
    :: k :: 수렴 기준. 0.000001 정도로 잡아 본다."""
    sum_xn = sum(network['x_n']) # 분모
    sum_varx = sum((network['x_n1'] - network['x_n']) ** 2) # 분자
    
    criterion = sum_varx / sum_xn
        
    if criterion <= k: # 수렴기준 충족
        return True
    
    else:
        return False

In [88]:
convergence = convergence_test(net_cost, 0.000001) # False가 저장될 시, 위 과정을 반복한다.
convergence

In [ ]:
net_cost['x_n'] = net_cost['x_n1']
net_cost['y_n'] = 0 # 초기화해주지 않으면 엄청 늘어나버린다.
net_cost['Direction'] = 0